Tensorflow를 이용한 머신러닝:
- 폐암 수술 환자의 생존율 예측
- 주어진 데이터를 이용해폐암 수술 환자의 생존율을 예측하는 프로그램을 만들어 보자.

- 사용 데이터: 폴란드 브로츠와프 의과대학의 2013년 폐암 수술 환자 데이터.
- 수술 전 진단 데이터와 수술 후 생존 결과를 기록한 의료기록으로 이루어져 있으며,470명의 환자, 18개 항목(1~17: 환자 정보, 18: 생존여부(1: 생존, 0: 사망))으로 이루어져 있다.

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
seed = 12345 # 실행 시점에 따라서 학습 결과가 달라지는 것을 방지하기 위함

In [3]:
data = np.loadtxt(r'C:\Users\user\Desktop\bigdata_8\DL\Day4\ThoraricSurgery.csv', delimiter=",")

In [4]:
data

array([[293.  ,   1.  ,   3.8 , ...,   0.  ,  62.  ,   0.  ],
       [  1.  ,   2.  ,   2.88, ...,   0.  ,  60.  ,   0.  ],
       [  8.  ,   2.  ,   3.19, ...,   0.  ,  66.  ,   1.  ],
       ...,
       [406.  ,   6.  ,   5.36, ...,   0.  ,  62.  ,   0.  ],
       [ 25.  ,   8.  ,   4.32, ...,   0.  ,  58.  ,   1.  ],
       [447.  ,   8.  ,   5.2 , ...,   0.  ,  49.  ,   0.  ]])

In [5]:
# X: 환자 기록, y: 수술 결과
X = data[:,:-1]
y = data[:,-1]

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, input_dim = 17, activation = 'relu'), # 노드 개수 30개, input feature 17개
    tf.keras.layers.GaussianNoise(.3), # 노이즈를 비율(0~1)로 넣는것. 모든 데이터에 정규분포 기반의 노이즈를 추가함.
                                       # 노이즈 추가는 과적합을 방지하는데 탁월하지만, 이미지의 미세한 차이는 사라질 수 있음.
                                       # 데이터의 편차가 클때 사용함.
    tf.keras.layers.Dense(16, activation='relu'), 
    tf.keras.layers.Dropout(.5), # 한 번의 학습 과정에서 전체 모델의 지정 비율을 사용하지 않음(과적합 방지)
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(.5),
    tf.keras.layers.Dense(1, activation = 'sigmoid') # sigmoid 함수를 이용한 이진 판별
])

c:\ProgramData\anaconda3\envs\DL\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
# 모델의 형태를 확인합니다.
# 모델의 깊이가 클수록 성능이 좋은 경향이 있음
# 필요 이상의 모델 복잡도를 피하기 위해서 param을 제한할 필요가 있음.
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gaussian_noise (GaussianNoise)  │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,753 (10.75 KB)

 Trainable params: 2,753 (10.75 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
model.compile(optimizer = 'adam',
              loss = 'mean_squared_error', # 손실함수는 평균제곱오차 함수 
              metrics=['accuracy'])

In [10]:
np.random.seed(seed)
tf.random.set_seed(seed)
model.fit(X, y, epochs = 50, batch_size = 10)

Epoch 1/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 780us/step - accuracy: 0.7308 - loss: 0.2526 
Epoch 2/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 642us/step - accuracy: 0.7943 - loss: 0.1975
Epoch 3/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 544us/step - accuracy: 0.8585 - loss: 0.1531
Epoch 4/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 522us/step - accuracy: 0.8332 - loss: 0.1780
Epoch 5/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 522us/step - accuracy: 0.8622 - loss: 0.1361
Epoch 6/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 522us/step - accuracy: 0.8491 - loss: 0.1564
Epoch 7/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 511us/step - accuracy: 0.8504 - loss: 0.1381
Epoch 8/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 587us/step - accuracy: 0.8655 - loss: 0.1415
Epoch 9/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 549us/step - accuracy: 0.8634 - loss: 0.1338
Epoch 10/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 511us/step - accuracy: 0.8634 - loss: 0.1452
Epoch 11/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 522us/step - accuracy: 0.8588 - loss: 0.1413
Epoch 12/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 500us/ste

## 학습결과 확인하기

In [10]:
model.evaluate(X, y)

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 643us/step - accuracy: 0.8366 - loss: 0.1382


[0.1267085075378418, 0.8510638475418091]

In [11]:
X[5]

array([18.  ,  2.  ,  2.96,  1.67,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  , 12.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  , 61.  ])

In [12]:
model.predict(X[5:6])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


array([[0.15007706]], dtype=float32)

In [13]:
y[5]

0.0